In [ ]:
# The following is from "big run"
import sys
sys.path.append('..')
from lib import *
import pandas as pd
import numpy as np

# Setup
lsa_np = np.load('../data/parsed/lsa_output.npy')
metadata = pd.read_pickle('../data/parsed/pickles/pickled_data_test.pickle')
metadata = metadata.loc[metadata['Scenario'] == '401']
metadata = metadata.reset_index(drop=True)
lsa_df = pd.DataFrame(lsa_np)
df = pd.concat([metadata, lsa_df], axis=1, join_axes=[metadata.index])
df = df.loc[df['Label'] != '-1']
df = df.reset_index(drop=True)
cat_features = ['To','From']
features = list(range(100))
features.extend(cat_features + ['Date'])
# features.extend(cat_features + ['ID'])

df = df[features + ['Label'] + ['ID']]
# df = df[features + ['ID']]

In [ ]:
# TODO: test tree behavior in ml_master when given .2 of training set to work with
# TODO: rather than making each tree its own process, divide the number of trees by the number of cores, and create
#       a list of trees that can be processed linearly in its own core.s
# TODO: I think multiprocessing will need to be done at the node level. 
#       Forest.fit multiprocesses to call tree.fit for each tree
#       tree.fit just calls self.head.split
#       each splitting node creates a pool of size two, for left and right. (recursively)

num_trees = 64
tries = 1
num_increments = 999
random_seed = 600
random_seed = None
overall_training_ratio = .8
initial_training_ratio = .2
increment_size = 10
cross_val_rnf_incremental(num_trees, df, tries, num_increments, random_seed,cat_features, overall_training_ratio,
                          initial_training_ratio, increment_size)

In [ ]:
forest = RNF(df, 1, 100, 100, 11, 100, [])
forest.fit()

In [ ]:
import time
start = time.time()
print(forest.predict(df[-100:], visualize=True))
end = time.time()
print(end - start)

In [ ]:
import time
start = time.time()
print(forest.predict(df[-100:], visualize=False))
end = time.time()
print(end - start)